In [118]:
import json
import csv
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
from config import api_key_NBA_fer

In [2]:
teamlist_id = [1,2,4,5,6,7,8,9,10,11,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,31,38,40,41]
teamlist_names = ["Atlanta Hawks","Boston Celtics","Brooklyn Nets","Charlotte Hornets","Chicago Bulls","Cleveland Cavaliers","Dallas Mavericks","Denver Nuggets","Detroit Pistons","Golden State Warriors","Houston Rockets","Indiana Pacers","LA Clippers","Los Angeles Lakers","Memphis Grizzlies","Miami Heat","Milwaukee Bucks","Minnesota Timberwolves","New Orleans Pelicans","New York Knicks","Oklahoma City Thunder","Orlando Magic","Philadelphia 76ers","Phoenix Suns","Portland Trail Blazers","Sacramento Kings","San Antonio Spurs","Toronto Raptors","Utah Jazz","Washington Wizards"]

In [156]:
teams_df = pd.DataFrame({
"Team Id": teamlist_id,
"Team Name": teamlist_names
})
teams_df.head()

,Team Id,Team Name
0,1,Atlanta Hawks
1,2,Boston Celtics
2,4,Brooklyn Nets
3,5,Charlotte Hornets
4,6,Chicago Bulls


In [38]:
team_details = []
team_url = "https://api-nba-v1.p.rapidapi.com/players/teamId"
for team in teams_df["Team Id"]:
    query_url = f"{team_url}/{team}"
    team_response = requests.get(query_url,headers={"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com","X-RapidAPI-Key": api_key_NBA_fer})
    print(query_url, team_response)
    teams_json = team_response.json()
    team_details.append(teams_json)

https://api-nba-v1.p.rapidapi.com/players/teamId/1 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/2 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/4 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/5 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/6 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/7 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/8 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/9 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/10 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/11 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/14 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/15 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/16 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/teamId/17 <Response [200]>
https://api-nba-v1.p.rapidapi.com/players/

In [150]:
team_details[0]["api"]["players"][0]

{'firstName': 'Justin',
 'lastName': 'Anderson',
 'teamId': '1',
 'yearsPro': '4',
 'collegeName': 'Virginia',
 'country': 'USA',
 'playerId': '17',
 'dateOfBirth': '1993-11-19',
 'affiliation': 'Virginia/USA',
 'startNba': '2015',
 'heightInMeters': '1.98',
 'weightInKilograms': '104.3',
 'leagues': {'standard': {'jersey': '1', 'active': '1', 'pos': 'G-F'}}}

In [63]:
player_list_id = []
player_list_first_name = []
player_list_last_name = []
positions_list = []
years_list = []

for team in team_details:
    for player in team["api"]["players"]:
        player_list_id.append(player["playerId"])
        player_list_first_name.append(player["firstName"])
        player_list_last_name.append(player["lastName"])
        years_list.append(player["yearsPro"])
        try:
            positions_list.append(player["leagues"]["standard"]["pos"])
        except KeyError:
            positions_list.append("N/A")

In [151]:
df = pd.DataFrame({
"Player Id": player_list_id,
"Name": player_list_first_name,
"Last Name": player_list_last_name,
"Years Pro": years_list,
"Position": positions_list
})
df = df.loc[df["Position"] != "N/A"]
df.head()

,Player Id,Name,Last Name,Years Pro,Position
0,17,Justin,Anderson,4,G-F
1,49,DeAndre',Bembry,3,G-F
2,94,Vince,Carter,21,G-F
3,119,Allen,Crabbe,6,G-F
4,133,Malcolm,Delaney,2,G


In [ ]:
player_data = []
counter = 1
for player in df["Player Id"]:
    player_response = requests.get(f"https://api-nba-v1.p.rapidapi.com/statistics/players/playerId/{player}", headers={"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com","X-RapidAPI-Key": api_key_NBA_fer})
    print(f"Processing player {counter}, result: {player_response}")
    counter = counter + 1
    player_json = player_response.json()
    player_data.append(player_json)

In [103]:
season_response = requests.get("https://api-nba-v1.p.rapidapi.com/games/seasonYear/2018",
  headers={
    "X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com",
    "X-RapidAPI-Key": api_key_NBA_fer
  }
)

In [104]:
season_details = season_response.json()
season_details["api"]["results"]

1868

In [105]:
games = []
leagues = []
for g in season_details["api"]["games"]:
    games.append(g["gameId"])
    leagues.append(g["league"])

In [106]:
games = pd.to_numeric(games)

In [107]:
games_df = pd.DataFrame({
"Game": games,
"League": leagues
})
games_df = games_df.loc[games_df["League"] == "standard"]
games_df.head()

,Game,League
95,4308,standard
96,4309,standard
97,4310,standard
98,4311,standard
99,4312,standard


In [108]:
player_data[0]["api"]["statistics"][0]

{'gameId': '15',
 'teamId': '8',
 'points': '1',
 'pos': 'F',
 'min': '22:20',
 'fgm': '0',
 'fga': '4',
 'fgp': '0.0',
 'ftm': '1',
 'fta': '2',
 'ftp': '50.0',
 'tpm': '0',
 'tpa': '4',
 'tpp': '0.0',
 'offReb': '0',
 'defReb': '1',
 'totReb': '1',
 'assists': '0',
 'pFouls': '1',
 'steals': '0',
 'turnovers': '2',
 'blocks': '1',
 'plusMinus': '-14',
 'playerId': '17'}

In [109]:
game_ids = []
player_ids = []
plusminus = []
minutes = []

for o in player_data:
    for f in o["api"]["statistics"]:
        f["gameId"] = pd.to_numeric(f["gameId"])
        if f["gameId"] in games_df["Game"]:
            game_ids.append(f["gameId"])
            player_ids.append(f["playerId"])
            plusminus.append(f["plusMinus"])
            minutes.append(f["min"])

In [110]:
df3 = pd.DataFrame({
"Game": game_ids,
"Player Id": player_ids,
"+/-": plusminus,
"Min": minutes
})
df3.head()

,Game,Player Id,+/-,Min
0,96,17,-11,26:51
1,106,17,-4,17:42
2,123,17,-11,3:28
3,129,17,3,18:22
4,166,17,2,0:42


In [111]:
df3[["Min", "Seg"]] = df3["Min"].str.split(":", expand = True)

In [112]:
df3["+/-"] = pd.to_numeric(df3["+/-"])
df3["Min"] = pd.to_numeric(df3["Min"])
df3["Seg"] = pd.to_numeric(df3["Seg"])
total_minutes = df3["Min"] + df3["Seg"]/60
df3["Minutes"] = total_minutes
weighed_plusminus = df3["+/-"]*df3["Minutes"]

In [113]:
df3["Minutes"] = total_minutes
df3["Weighed +/-"] = weighed_plusminus
df3.head()

,Game,Player Id,+/-,Min,Seg,Minutes,Weighed +/-
0,96,17,-11.0,26.0,51.0,26.850000,-295.350000
1,106,17,-4.0,17.0,42.0,17.700000,-70.800000
2,123,17,-11.0,3.0,28.0,3.466667,-38.133333
3,129,17,3.0,18.0,22.0,18.366667,55.100000
4,166,17,2.0,0.0,42.0,0.700000,1.400000


In [114]:
grouped_df3 = df3.groupby(["Player Id"])
grouped_players = grouped_df3["Game"].count()
grouped_plusminus = grouped_df3["+/-"].mean()
grouped_minutes = grouped_df3["Minutes"].sum()
grouped_weighed = grouped_df3["Weighed +/-"].sum()

In [115]:
df5 = pd.DataFrame({
"Games Played": grouped_players,
"Average +/-": grouped_plusminus,
"Minutes Played": grouped_minutes,
"Total Weighed +/-": grouped_weighed
})
df5.head()

,Games Played,Average +/-,Minutes Played,Total Weighed +/-
Player Id,,,,
101,93,-5.012195,2037.266667,-8781.133333
102,29,-0.222222,792.750000,-257.816667
104,28,-4.535714,501.816667,-2216.100000
108,131,-0.432432,1134.533333,121.216667
109,113,-6.433628,3469.216667,-21749.333333


In [128]:
df6 = pd.merge(df, df5, on = "Player Id")
df6.head()

,Player Id,Name,Last Name,Years Pro,Position,Games Played,Average +/-,Minutes Played,Total Weighed +/-
0,17,Justin,Anderson,4,G-F,108,-1.780220,1325.633333,-3092.316667
1,49,DeAndre',Bembry,3,G-F,27,0.666667,155.333333,291.650000
2,94,Vince,Carter,21,G-F,117,-0.586207,1657.083333,-295.550000
3,119,Allen,Crabbe,6,G-F,124,-0.709677,3285.183333,-1742.533333
4,133,Malcolm,Delaney,2,G,30,2.275862,483.466667,1343.966667


In [152]:
df6["Factor"] = df6["Total Weighed +/-"]/df6["Games Played"]
df6.sort_values(by = "Factor", ascending = False).head()

,Player Id,Name,Last Name,Years Pro,Position,Games Played,Average +/-,Minutes Played,Total Weighed +/-,Factor,Full Name
126,204,Draymond,Green,7,F,138,11.873134,4599.950000,52312.250000,379.074275,Draymond Green
125,124,Stephen,Curry,10,G,132,11.961240,4310.883333,49092.716667,371.914520,Stephen Curry
133,514,Klay,Thompson,8,G,138,9.338235,4509.416667,40244.033333,291.623430,Klay Thompson
28,153,Kevin,Durant,12,F,124,9.016529,4294.350000,34652.866667,279.458602,Kevin Durant
190,265,LeBron,James,16,F,127,8.380165,4335.966667,34829.250000,274.246063,LeBron James


In [130]:
df6["Full Name"] = df6["Name"] + " " + df6["Last Name"]
df6.head()

,Player Id,Name,Last Name,Years Pro,Position,Games Played,Average +/-,Minutes Played,Total Weighed +/-,Factor,Full Name
0,17,Justin,Anderson,4,G-F,108,-1.780220,1325.633333,-3092.316667,-28.632562,Justin Anderson
1,49,DeAndre',Bembry,3,G-F,27,0.666667,155.333333,291.650000,10.801852,DeAndre' Bembry
2,94,Vince,Carter,21,G-F,117,-0.586207,1657.083333,-295.550000,-2.526068,Vince Carter
3,119,Allen,Crabbe,6,G-F,124,-0.709677,3285.183333,-1742.533333,-14.052688,Allen Crabbe
4,133,Malcolm,Delaney,2,G,30,2.275862,483.466667,1343.966667,44.798889,Malcolm Delaney


In [133]:
NBA_data = "nba_data.csv"
df7 = pd.read_csv(NBA_data, sep = ";")
df7["Full Name"] = df7["Name"]
df7.head()

,Name,NBA Salary,Age,S%,Full Name
0,Stephen Curry,37457154,31.3,100,Stephen Curry
1,LeBron James,35654150,34.5,100,LeBron James
2,Chris Paul,35654150,34.2,100,Chris Paul
3,Russell Westbrook,35654150,30.7,100,Russell Westbrook
4,James Harden,35650150,29.9,100,James Harden


In [158]:
df8 = pd.merge(df6, df7, on = "Full Name")
df8 = df8[["Player Id", "Full Name", "Age", "Years Pro", "Position", "Factor", "NBA Salary"]]
df8.head(3)

,Player Id,Full Name,Age,Years Pro,Position,Factor,NBA Salary
0,17,Justin Anderson,25.7,4,G-F,-28.632562,2516048
1,49,DeAndre' Bembry,25.0,3,G-F,10.801852,1634640
2,94,Vince Carter,42.5,21,G-F,-2.526068,1512601


In [155]:
df8.to_csv("NBA_output.csv")